In [10]:
import sys
from pathlib import Path

# Add the parent directory to the path
# sys.path.append(str(Path(__file__).parent.parent))

from guess.charge3net_core.src.real_predictor import load_real_predictor
from guess.charge3net_core.src.convert_chgcar_to_pkl import ChgcarToPklConverter
from guess.charge3net_core.src.convert_pkl_to_chgcar import PklToChgcarConverter
from guess import get_package_data_path

In [5]:

print("Step 1: Converting CHGCAR to pickle format...")
chgcar_to_pkl = ChgcarToPklConverter()
chgcar_to_pkl.convert_file(
    chgcar_file="./examples/example_c3n/natoms/chgcar.10",
    output_file="./data/natoms/10/10",
    overwrite=True,
    spin=False
)

Step 1: Converting CHGCAR to pickle format...


In [11]:
print("Loading real predictor...")
predictor = load_real_predictor(
            checkpoint_path=f"{get_package_data_path('.')}/charge3net_core/models/charge3net_mp.pt",
            config_path=f"{get_package_data_path('.')}/charge3net_core/configs/charge3net/test_chgcar_inputs.yaml",
            device='cpu'
        )
        
model_info = predictor.get_model_info()
print(f"✓ Real predictor loaded successfully:")


Loading real predictor...
Failed to instantiate components from config: 'DictConfig' object is not callable
Using default components...
✓ Real predictor loaded successfully:


In [ ]:
# Make real predictionprint("\nMaking real prediction with actual data loading...")
predictor.max_predict_batch_probes=500
result = predictor.predict_single_real(
            density_file="./data/natoms/2/2.npy",
            atoms_file="./data/natoms/2/2_atoms.pkl",
            input_path='./data/natoms/2/inputs/',
            output_file="./data/output/2_prediction.npy",
            return_cube=True
        )
        
print(f"✓ Real prediction completed:")
print(f"  - Filename: {result['filename']}")
print(f"  - Grid shape: {result['grid_shape']}")
print(f"  - Prediction time: {result['prediction_time']:.3f}s")
print(f"  - Note: {result['note']}")

Making real prediction for 2.npy
Density shape: (24, 24, 24)
Atoms: 2 atoms
Setting up datamodule for input_dir: data/natoms/2/inputs
Datamodule created successfully: <class 'guess.charge3net_core.src.charge3net.data.dataset.DensityDatamodule'>
Datamodule has test_dataloader: True
✓ Full datamodule setup completed successfully
✓ Real prediction completed in 34.723s
✓ Real prediction completed:
  - Filename: density
  - Grid shape: [24 24 24]
  - Prediction time: 34.723s
  - Note: Real prediction using actual data loading and graph construction


In [ ]:
natoms=10

print("Step 1: Converting CHGCAR to pickle format...")
chgcar_to_pkl = ChgcarToPklConverter()
chgcar_to_pkl.convert_file(
    chgcar_file=f"./examples/example_c3n/natoms/chgcar.{natoms}",
    output_file=f"./data/natoms/{natoms}/{natoms}",
    overwrite=True,
    spin=False
)
predictor.max_predict_batch_probes=500
result = predictor.predict_single_real(
            density_file=f"./data/natoms/{natoms}/{natoms}.npy",
            atoms_file=f"./data/natoms/{natoms}/{natoms}_atoms.pkl",
            input_path=f'./data/natoms/{natoms}/inputs/',
            output_file=f"./data/output/{natoms}_prediction.npy",
            return_cube=True
        )
        

print(f"✓ Real prediction completed:")
print(f"  - Filename: {result['filename']}")
print(f"  - Grid shape: {result['grid_shape']}")
print(f"  - Prediction time: {result['prediction_time']:.3f}s")
print(f"  - Note: {result['note']}")

Making real prediction for 10.npy
Density shape: (24, 24, 108)
Atoms: 10 atoms
Setting up datamodule for input_dir: data/natoms/10/inputs
Datamodule created successfully: <class 'guess.charge3net_core.src.charge3net.data.dataset.DensityDatamodule'>
Datamodule has test_dataloader: True
✓ Full datamodule setup completed successfully
✓ Real prediction completed in 127.179s
✓ Real prediction completed:
  - Filename: density
  - Grid shape: [ 24  24 108]
  - Prediction time: 127.179s
  - Note: Real prediction using actual data loading and graph construction


In [4]:
import torch
if torch.backends.mps.is_available():
    torch.mps.empty_cache()

In [ ]:
print("CHGCAR → Pickle → Real Prediction → CHGCAR")

print("Step 1: Converting CHGCAR to pickle format...")
chgcar_to_pkl = ChgcarToPklConverter()
chgcar_to_pkl.convert_file(
    chgcar_file="./data/sample_inputs/si.CHGCAR",
    output_file="./data/sample_inputs/si_real_pipeline",
    overwrite=True,
    spin=False
)
print("✓ CHGCAR to pickle conversion completed")

# Step 2: Load real predictor
print("\nStep 2: Loading real predictor...")
predictor = load_real_predictor(
    checkpoint_path="./models/charge3net_mp.pt",
    config_path="./configs/charge3net/test_chgcar_inputs.yaml"
)
print("✓ Real predictor loaded successfully")

# Step 3: Make real prediction
print("\nStep 3: Making real prediction with actual data loading...")
result = predictor.predict_single_real(
    density_file="./data/sample_inputs/si_real_pipeline.npy",
    atoms_file="./data/sample_inputs/si_real_pipeline_atoms.pkl",
    output_file="./predictions/si_real_pipeline_prediction.npy",
    return_cube=True
)

print(f"✓ Real prediction completed in {result['prediction_time']:.3f}s")
print(f"  - Note: {result['note']}")

# Step 4: Convert prediction back to CHGCAR
print("\nStep 4: Converting real prediction to CHGCAR format...")
pkl_to_chgcar = PklToChgcarConverter()
pkl_to_chgcar.convert_and_save(
    density_file="./predictions/si_real_pipeline_prediction.npy",
    atoms_file="./data/sample_inputs/si_real_pipeline_atoms.pkl",
    output_file="./predictions/Si_real_pipeline_prediction.CHGCAR"
)
print("✓ Real prediction converted to CHGCAR format")

# Summary
print("\n" + "=" * 50)
print(" Real pipeline completed successfully!")
print("\nOutput files:")
print(f"  - Pickle files: ./data/sample_inputs/si_real_pipeline.npy/.pkl")
print(f"  - Prediction: ./predictions/si_real_pipeline_prediction.npy")
print(f"  - CHGCAR: ./predictions/Si_real_pipeline_prediction.CHGCAR")


In [ ]:
times=[]
natoms_list=[2,10,20,50,100]
for natoms in natoms_list:
  print("="*10)
  chgcar_to_pkl = ChgcarToPklConverter()
  chgcar_to_pkl.convert_file(
      chgcar_file=f"./examples/example_c3n/natoms/chgcar.{natoms}",
      output_file=f"./data/natoms/{natoms}/{natoms}",
      overwrite=True,
      spin=False
  )
  predictor.max_predict_batch_probes=500
  result = predictor.predict_single_real(
              density_file=f"./data/natoms/{natoms}/{natoms}.npy",
              atoms_file=f"./data/natoms/{natoms}/{natoms}_atoms.pkl",
              input_path=f'./data/natoms/{natoms}/inputs/',
              output_file=f"./data/output/{natoms}_prediction.npy",
              return_cube=True
          )
  torch.cuda.empty_cache()
  times.append(result['prediction_time'])

  print(f"✓ Real prediction completed:")
  print(f"  - Filename: {result['filename']}")
  print(f"  - Grid shape: {result['grid_shape']}")
  print(f"  - Prediction time: {result['prediction_time']:.3f}s")
  print(f"  - Note: {result['note']}")

  pkl_to_chgcar = PklToChgcarConverter()
  pkl_to_chgcar.convert_and_save(
      density_file=f"./data/output/{natoms}_prediction.npy",
      atoms_file=f"./data/natoms/{natoms}/{natoms}_atoms.pkl",
      output_file=f"/content/ElDensiago///data/output/{natoms}_prediction.CHGCAR"
  )
  print("✓ Real prediction converted to CHGCAR format")
  print('='*10)

